In [2]:
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox
import numpy as np
from pytket.circuit import OpType

# u1 = np.asarray([[1, 0],
#                  [0, 1]])
# u1box = Unitary1qBox(u1)

#U1 - diagnol matrix where input_angle / 2 = eigen-value = phase estimate [0,1) 
#we easily check this 
input_angle = 0.73  # angle as number of half turns
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops

In [3]:
from pytket.circuit import Op

base_u1 = Op.create(OpType.U1, [0.5])

In [4]:
from pytket.circuit import QControlBox
from pytket.circuit import CircBox

controlled_u1 = QControlBox(CircBox(unitary_circuit), n=2)

In [6]:
from pytket import Circuit
from pytket import OpType
from pytket.circuit.display import render_circuit_jupyter

basic_circuit = Circuit(2,1)

basic_circuit.H(0)
basic_circuit.X(1)
basic_circuit.CU1(input_angle, 0, 1)
basic_circuit.H(0)

#After second H-gate we measure and save to classical bit and read that value
basic_circuit.Measure(0,0)

render_circuit_jupyter(basic_circuit)


basic_circuit.name = "iterative_basic_arjun"


In [7]:
from pytket.extensions.nexus import Nexus
from datetime import datetime

nexus = Nexus()
my_project = nexus.new_project(name=f"Nexus iQuHack tutorial project! - {datetime.now()}")


New project created: Nexus iQuHack tutorial project! - 2024-02-04 00:14:34.774887


In [8]:
from pytket.extensions.nexus import QuantinuumConfig

#user_group="iQuHACK_2024" <- only use on H1-1LE emulator
configuration = QuantinuumConfig(device_name="H1-1E")

In [9]:
from pytket.extensions.nexus import NexusBackend

# Then we'll create a NexusBackend using our config and the project we created
backend = NexusBackend(configuration, project=my_project)


Started using project with name: Nexus iQuHack tutorial project! - 2024-02-04 00:14:34.774887


In [10]:
# Compile the circuit in Nexus
compiled_circuit = backend.get_compiled_circuit(basic_circuit, optimisation_level=2)

In [11]:
# Run the compiled circuit for 10 shots
handle = backend.process_circuit(compiled_circuit, n_shots=50)


# This will give you a ResultHandle
# As with other pytket-backends, a ResultHandle identifies a particular run of the circuit, which you can then use to keep track of the job status.
print(handle)

('cfa0e60f-323f-4600-b18a-a42a2c0f5c39', 1894666)


In [12]:
# Get an update on the status
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message='Circuit has been submitted to Nexus for Processing.', error_detail=None, completed_time=None, queued_time=None, submitted_time=datetime.datetime(2024, 2, 4, 0, 14, 51, 584504, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [15]:
# If the job is COMPLETED we can then retrieve the result with the handle
result = backend.get_result(handle)
result.get_counts()

Counter({(0,): 12, (1,): 38})

In [19]:
result.get_distribution()

{(0,): 0.24, (1,): 0.76}

In [26]:
import math

theta_estimate = math.acos(math.sqrt(0.24)) / math.pi

theta_estimate

0.3370340319377619

In [29]:
input_angle / 2

0.365